In [1]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [2]:
# !pip install stable-baselines3\[extra\]
# !pip install gym

In [13]:
import os 
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [27]:
environment_name = "CartPole-v0"

In [15]:
# env = gym.make(environment_name, render_mode = "human")
environment_name


'CartPole-v0'

In [28]:
environment_name = "CartPole-v0"
env = gym.make(environment_name, render_mode = "human")
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, trunc, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:37.0
Episode:2 Score:25.0
Episode:3 Score:18.0
Episode:4 Score:14.0
Episode:5 Score:18.0


In [29]:
env.close()
# env.reset()

In [30]:
episodes = 5
for episode in range(1,episodes+1):
    print(episode)

1
2
3
4
5


In [32]:
# env.reset()

In [33]:
# env.step(1)

error: display Surface quit

# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [43]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [44]:
env.action_space

Discrete(2)

In [45]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-1.5771471e+00, -2.4608506e+38, -8.6691692e-02, -8.6396635e+37],
      dtype=float32)

# 3. Train an RL Model

In [46]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
# model = PPO('MlpPolicy', env, verbose = 1)
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [56]:
model.learn(total_timesteps=20000) # want to train it longer, run this again

Logging to Training/Logs/PPO_7
-----------------------------
| time/              |      |
|    fps             | 6602 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 3122         |
|    iterations           | 2            |
|    time_elapsed         | 1            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0013363163 |
|    clip_fraction        | 0.013        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.342       |
|    explained_variance   | -0.682       |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00174     |
|    n_updates            | 510          |
|    policy_gradient_loss | 0.000151     |
|    value_loss           | 4.39e-06     |
----------------------------

In [53]:
log_path = os.path.join('Training','Logs') # run these lines before the one above

In [54]:
log_path

'Training/Logs'

# 4. Save and Reload Model

In [57]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [59]:
model.save(PPO_path)

In [60]:
del model

In [65]:
# model = PPO.load('PPO_model', env=env)
model = PPO.load(PPO_path, env=env)


# 4. Evaluation

In [68]:
from stable_baselines3.common.evaluation import evaluate_policy

In [71]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [72]:
env.close()

In [75]:
# 6. test model
# action, _= model.predict(obs)


NameError: name 'obs' is not defined

# 5. Test Model

In [76]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.00750508, -0.03663883, -0.00260175,  0.04928632], dtype=float32)}]


In [77]:
env.close()

# 6. Viewing Logs in Tensorboard

In [78]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [ ]:
!tensorboard --logdir={training_log_path}

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.13.0 at http://localhost:6006/ (Press CTRL+C to quit)


# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()